In [16]:
# import packages
import pandas as pd
from pandas import json_normalize

In [17]:
file_paths = {
    'Bangalore': 'Raw_Datasets/bangalore_cars.xlsx',
    'Chennai': 'Raw_Datasets/chennai_cars.xlsx',
    'Delhi': 'Raw_Datasets/delhi_cars.xlsx',
    'Hyderabad': 'Raw_Datasets/hyderabad_cars.xlsx',
    'Jaipur': 'Raw_Datasets/jaipur_cars.xlsx',
    'Kolkata': 'Raw_Datasets/kolkata_cars.xlsx'
}

In [18]:
dataframes = []

def add_city_and_concatenate(file_paths):
    for city,path in file_paths.items():
        df = pd.read_excel(path)
        df["city"] = city
        dataframes.append(df)
    return pd.concat(dataframes,ignore_index=True)

concatenated_df = add_city_and_concatenate(file_paths)


In [19]:
def convert_jsonstr_to_dict(df):
    columns_to_be_checked = ['new_car_overview', 'new_car_feature', 'new_car_specs']
    for column in columns_to_be_checked:
        df[column] = df[column].apply(lambda x: eval(x) if isinstance(x,str) else x)
    return df

In [20]:
def format_new_car_overview(record):
    temp_dict = {}
    top_items = record.get("top",[])
    for item in top_items:
        key = item.get("key")
        value = item.get("value")
        temp_dict[key] = value
    return temp_dict

In [21]:
def format_new_car_feature(record):
    temp_list = []
    top_items = record.get("top", [])
    for item in top_items:
        temp_list.append(item['value'])

    data_items = record.get("data", [])
    for item in data_items:
        sub_items = item.get("list", [])
        for sub_item in sub_items:
            temp_list.append(sub_item['value'])

    return temp_list


In [22]:
def format_new_car_specs(record):
    temp_dict = {}

    top_items = record.get("top",[])
    for item in top_items:
        if 'key' in item and 'value' in item:
            temp_dict[item['key']] = item['value']

    data_items = record.get("data",[])
    for item in data_items:
        sub_items = item.get("list",[])
        for item in sub_items:
            if 'key' in item and 'value' in item:
                temp_dict[item['key']] = item['value']

    return temp_dict

In [23]:
def apply_data_extraction_to_columns(formatted_df):
    formatted_df['new_car_detail'] = formatted_df['new_car_detail'].apply(lambda x: eval(x) if isinstance(x,str) else x)
    formatted_df['new_car_overview'] = formatted_df['new_car_overview'].apply(lambda x: format_new_car_overview(x) if isinstance(x,dict) else x)
    formatted_df['new_car_feature'] = formatted_df['new_car_feature'].apply(lambda x: format_new_car_feature(x) if isinstance(x,dict) else x)
    formatted_df['new_car_specs'] = formatted_df['new_car_specs'].apply(lambda x: format_new_car_specs(x) if isinstance(x,dict) else x)
    return formatted_df

In [24]:
def main():
    # Step 1: Concatenate data from multiple file paths and add city information
    concatenated_df = add_city_and_concatenate(file_paths)
    
    # Step 2: Convert JSON strings in concatenated_df to dictionaries
    formatted_df = convert_jsonstr_to_dict(concatenated_df)
    
    # Step 3: Apply data extraction to specific columns
    extracted_df = apply_data_extraction_to_columns(formatted_df)

    # Step 4: Flatten each nested column and create separate DataFrames
    car_details = pd.DataFrame(extracted_df["new_car_detail"].tolist())
    car_overview = pd.DataFrame(extracted_df["new_car_overview"].tolist())
    car_specs = pd.DataFrame(extracted_df["new_car_specs"].tolist())

    # Step 5: Drop the original nested columns from extracted_df
    extracted_df = extracted_df.drop(columns=["new_car_detail", "new_car_overview", "new_car_specs"])

    # Step 6: Concatenate the flattened DataFrames with extracted_df horizontally
    flattened_df = pd.concat([extracted_df, car_details, car_overview, car_specs], axis=1)

    # Return the final flattened DataFrame for further use
    return flattened_df


In [25]:
df = main()

KeyboardInterrupt: 

In [ ]:
df.head(2)

,new_car_feature,car_links,city,it,ft,bt,km,transmission,ownerNo,owner,...,Front Brake Type,Rear Brake Type,Top Speed,Acceleration,Tyre Type,No Door Numbers,Cargo Volumn,Wheel Size,Alloy Wheel Size,Ground Clearance Unladen
0,"[Power Steering, Power Windows Front, Air Cond...",https://www.cardekho.com/used-car-details/used...,Bangalore,0,Petrol,Hatchback,"1,20,000",Manual,3,3rd Owner,...,Ventilated Disc,Drum,150 Kmph,15.05 Seconds,"Tubeless, Radial",5,235-litres,NaN,NaN,NaN
1,"[Power Steering, Power Windows Front, Air Cond...",https://www.cardekho.com/buy-used-car-details/...,Bangalore,0,Petrol,SUV,"32,706",Manual,2,2nd Owner,...,Ventilated Disc,Drum,NaN,NaN,"Tubeless,Radial",4,352-litres,16,16,NaN


In [ ]:
df.columns

Index(['new_car_feature', 'car_links', 'city', 'it', 'ft', 'bt', 'km',
       'transmission', 'ownerNo', 'owner', 'oem', 'model', 'modelYear',
       'centralVariantId', 'variantName', 'price', 'priceActual',
       'priceSaving', 'priceFixedText', 'trendingText', 'Registration Year',
       'Insurance Validity', 'Fuel Type', 'Seats', 'Kms Driven', 'RTO',
       'Ownership', 'Engine Displacement', 'Transmission',
       'Year of Manufacture', 'Mileage', 'Engine', 'Max Power', 'Torque',
       'Seats', 'Color', 'Engine Type', 'Displacement', 'Max Torque',
       'No of Cylinder', 'Values per Cylinder', 'Value Configuration',
       'Fuel Suppy System', 'BoreX Stroke', 'Compression Ratio',
       'Turbo Charger', 'Super Charger', 'Length', 'Width', 'Height',
       'Wheel Base', 'Front Tread', 'Rear Tread', 'Kerb Weight',
       'Gross Weight', 'Gear Box', 'Drive Type', 'Seating Capacity',
       'Steering Type', 'Turning Radius', 'Front Brake Type',
       'Rear Brake Type', 'Top Speed'

In [ ]:
#Remove unwanted columns
unwanted_columns = [
    'new_car_feature', 'car_links', 'Insurance Validity', 'it', 'ft', 'bt', 'km', 'owner', 'centralVariantId', 
    'priceActual', 'priceSaving', 'priceFixedText', 'trendingText', 'Seats', 
    'RTO', 'Engine', 'Max Power', 'Torque', 'Color', 'Engine Type', 
    'Displacement', 'Max Torque', 'No of Cylinder', 'Values per Cylinder', 
    'Value Configuration', 'Fuel Suppy System', 'BoreX Stroke', 'Compression Ratio', 
    'Turbo Charger', 'Super Charger', 'Length', 'Width', 'Height', 'Wheel Base', 
    'Front Tread', 'Rear Tread', 'Kerb Weight', 'Gross Weight', 'Gear Box', 
    'Seating Capacity', 'Steering Type', 'Turning Radius', 'Front Brake Type', 
    'Rear Brake Type', 'Top Speed', 'Acceleration', 'Tyre Type', 'No Door Numbers', 
    'Cargo Volumn', 'Wheel Size', 'Alloy Wheel Size', 'Ground Clearance Unladen','Drive Type','Ownership','Year of Manufacture','Transmission'
]

trimmed_df = df.drop(columns=unwanted_columns)

In [ ]:
trimmed_df.columns

Index(['city', 'transmission', 'ownerNo', 'oem', 'model', 'modelYear',
       'variantName', 'price', 'Registration Year', 'Fuel Type', 'Kms Driven',
       'Engine Displacement', 'Mileage'],
      dtype='object')

In [26]:
# Export it
trimmed_df.to_csv(r"SBS_Processed_Datasets/01_structured_output.csv", index=False)